# Carga Datos y Librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
import os

In [2]:
df=pd.read_csv("../data/raw/Propensity.csv")
df.head()

,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0
2,CLI3,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,...,Medio,NO DATA,No,NO,1376,0,True,4,21,3
3,CLI4,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,...,Medio,SI,Averia muy grave,SI,2015,7256,True,4,48,5
4,CLI5,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,...,Alto,NO DATA,No,NO,1818,0,True,3,21,3


# Conteo Valores Nulos

In [3]:
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_summary = pd.DataFrame({
    "Valores Nulos": df.isnull().sum(),
    "Porcentaje (%)": missing_percentage
})

missing_summary = missing_summary[missing_summary["Valores Nulos"] > 0]
missing_summary


,Valores Nulos,Porcentaje (%)
ESTADO_CIVIL,890,1.533187
GENERO,860,1.481507
Zona_Renta,13178,22.701511
Averia_grave,1,0.001723


# Propensity_clean_v1

Para este dataset vamos a aplicar las siguientes medidas
- Eliminar todos los valores nulos
- Transformar las variables categóricas

**Transformacion variables categóricas**

*Label Encoding:*

PRODUCTO, TIPO_CARROCERIA, COMBUSTIBLE, FORMA_PAGO, ESTADO_CIVIL, GENERO, OcupaciOn, PROVINCIA, Zona_Renta, REV_Garantia, QUEJA_CAC, Campanna1, Campanna2, Campanna3, Mas_1_coche,TRANS,REV_Garantia, Zona_Renta

*Ordinal Encoding:* 

Potencia, Averia_grave  



In [4]:
#eliminar valores nulos
df_clean_v1=df.dropna()

In [5]:
#eliminamos la columna CODE
df_clean_v1=df_clean_v1.drop(columns=['CODE'])

In [6]:
df_clean_v1.columns

Index(['PRODUCTO', 'TIPO_CARROCERIA', 'COMBUSTIBLE', 'Potencia', 'TRANS',
       'FORMA_PAGO', 'ESTADO_CIVIL', 'GENERO', 'OcupaciOn', 'PROVINCIA',
       'Campanna1', 'Campanna2', 'Campanna3', 'Zona_Renta', 'REV_Garantia',
       'Averia_grave', 'QUEJA_CAC', 'COSTE_VENTA', 'km_anno', 'Mas_1_coche',
       'Revisiones', 'Edad_Cliente', 'Tiempo'],
      dtype='object')

In [7]:
# Label Encoding:
label_cols = ["PRODUCTO", "TIPO_CARROCERIA", "COMBUSTIBLE", "FORMA_PAGO", "ESTADO_CIVIL", 
              "GENERO", "OcupaciOn", "PROVINCIA", "REV_Garantia", "QUEJA_CAC", 
              "Campanna1", "Campanna2", "Campanna3", "Mas_1_coche", "TRANS"]  

label_encoders = {}

for col in label_cols:
    le = LabelEncoder()
    df_clean_v1[col] = le.fit_transform(df_clean_v1[col])
    label_encoders[col] = le  # Guardamos el encoder por si necesitamos revertirlo

# Confirmar la transformación en algunas columnas
df_clean_v1[label_cols].head()


,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,REV_Garantia,QUEJA_CAC,Campanna1,Campanna2,Campanna3,Mas_1_coche,TRANS
0,0,0,0,0,0,1,1,4,0,1,1,0,0,0,1
1,0,0,0,0,0,0,1,46,1,0,0,0,0,0,1
2,0,0,0,3,0,1,1,29,0,0,0,0,0,1,1
3,0,0,0,2,0,0,1,31,1,1,1,0,0,1,1
4,0,0,0,2,0,0,2,40,0,0,1,0,1,1,1


In [8]:
#comprobamos los valores que hay
columns_of_interest = ["PRODUCTO","COMBUSTIBLE", "Potencia", "TRANS", "FORMA_PAGO", 
                       "ESTADO_CIVIL", "GENERO", "OcupaciOn", 
                       "Averia_grave", "Mas_1_coche",'Campanna1','QUEJA_CAC','Zona_Renta']
for col in columns_of_interest:
    print(f"Valores únicos en {col}:")
    print(df_clean_v1[col].unique())
    print("")

Valores únicos en PRODUCTO:
[ 0  1  2  7  9  3  8  4  5 10  6]

Valores únicos en COMBUSTIBLE:
[0 1]

Valores únicos en Potencia:
['Baja' 'Media' 'Alta']

Valores únicos en TRANS:
[1 0]

Valores únicos en FORMA_PAGO:
[0 3 2 1]

Valores únicos en ESTADO_CIVIL:
[0 3 2 1]

Valores únicos en GENERO:
[1 0]

Valores únicos en OcupaciOn:
[1 2 0]

Valores únicos en Averia_grave:
['Averia muy grave' 'No' 'Averia leve' 'Averia grave']

Valores únicos en Mas_1_coche:
[0 1]

Valores únicos en Campanna1:
[1 0]

Valores únicos en QUEJA_CAC:
[1 0]

Valores únicos en Zona_Renta:
['Medio-Bajo' 'Medio' 'Alto' 'Otros']



Aplicamos ordinal-encod para las variables que presentan un cierto orden como Averia_grave o Potencia

In [9]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories=[
    ["Baja", "Media", "Alta"],  # Potencia: 0, 1, 2
    ["No", "Averia leve", "Averia grave", "Averia muy grave"],  # Averia_grave: 0, 1, 2, 3
    ["Otros", "Medio-Bajo", "Medio", "Alto"]  # Zona_Renta: 'Otros' es la peor categoría (0)
])

# Aplicar Ordinal Encoding a 'Potencia', 'Averia_grave' y 'Zona_Renta' en df_clean_v1 y convertir a enteros
df_clean_v1[["Potencia", "Averia_grave", "Zona_Renta"]] = ordinal_encoder.fit_transform(
    df_clean_v1[["Potencia", "Averia_grave", "Zona_Renta"]]
).astype(int)


In [10]:
df_clean_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44119 entries, 0 to 58047
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   PRODUCTO         44119 non-null  int64
 1   TIPO_CARROCERIA  44119 non-null  int64
 2   COMBUSTIBLE      44119 non-null  int64
 3   Potencia         44119 non-null  int64
 4   TRANS            44119 non-null  int64
 5   FORMA_PAGO       44119 non-null  int64
 6   ESTADO_CIVIL     44119 non-null  int64
 7   GENERO           44119 non-null  int64
 8   OcupaciOn        44119 non-null  int64
 9   PROVINCIA        44119 non-null  int64
 10  Campanna1        44119 non-null  int64
 11  Campanna2        44119 non-null  int64
 12  Campanna3        44119 non-null  int64
 13  Zona_Renta       44119 non-null  int64
 14  REV_Garantia     44119 non-null  int64
 15  Averia_grave     44119 non-null  int64
 16  QUEJA_CAC        44119 non-null  int64
 17  COSTE_VENTA      44119 non-null  int64
 18  km_anno    

In [11]:
base_dir = os.path.abspath("..")  # Sube un nivel en la estructura de carpetas si ejecutas desde notebooks
processed_dir = os.path.join(base_dir, "data", "processed")
os.makedirs(processed_dir, exist_ok=True)  

df_clean_v1.to_csv(os.path.join(processed_dir, "Propensity_clean_v1.csv"), index=False)


# Propensity_clean_v2

Para este dataset vamos a aplicar las siguientes medidas
- Eliminar el nulo en `Averia_Grave`
- Sustituir `Estado_Civil` y `Genero` por la moda de estas variables
- En `Zona_Renta` los nulos->Desconocido y en el O-E le damos la menor importancia

In [12]:
df_clean_v2=df.drop(columns=['CODE'])

In [13]:
df_clean_v2 = df_clean_v2.dropna(subset=['Averia_grave'])

In [14]:
# Calcular la moda de las columnas ESTADO_CIVIL y GENERO en df_clean_v2
moda_estado_civil_v2 = df_clean_v2['ESTADO_CIVIL'].mode()[0]
moda_genero_v2 = df_clean_v2['GENERO'].mode()[0]

df_clean_v2['ESTADO_CIVIL'] = df_clean_v2['ESTADO_CIVIL'].fillna(moda_estado_civil_v2)
df_clean_v2['GENERO'] = df_clean_v2['GENERO'].fillna(moda_genero_v2)


In [15]:
# Sustituir valores nulos en 'Zona_Renta' con 'Desconocido'
df_clean_v2["Zona_Renta"] = df_clean_v2["Zona_Renta"].fillna("Desconocido")


In [16]:

label_cols = ["PRODUCTO", "TIPO_CARROCERIA", "COMBUSTIBLE", "FORMA_PAGO", "ESTADO_CIVIL", 
              "GENERO", "OcupaciOn", "PROVINCIA", "REV_Garantia", "QUEJA_CAC", 
              "Campanna1", "Campanna2", "Campanna3", "Mas_1_coche", "TRANS"]  

label_encoders_v2 = {}

for col in label_cols:
    le = LabelEncoder()
    df_clean_v2[col] = le.fit_transform(df_clean_v2[col])
    label_encoders_v2[col] = le  # Guardamos el encoder por si necesitamos revertirlo


In [17]:
#como hemos transformado los nulos por desconocido le vamos a dar el menor peso a desconocido
df_clean_v2['Zona_Renta'].unique()

array(['Medio-Bajo', 'Medio', 'Alto', 'Desconocido', 'Otros'],
      dtype=object)

In [18]:
#Ordinal Encoding: 'Potencia', 'Averia_grave' y 'Zona_Renta' en df_clean_v2


# Definir el orden correcto para cada variable
ordinal_encoder = OrdinalEncoder(categories=[
    ["Baja", "Media", "Alta"],  # Potencia: 0, 1, 2
    ["No", "Averia leve", "Averia grave", "Averia muy grave"],  # Averia_grave: 0, 1, 2, 3
    ["Desconocido", "Otros","Medio-Bajo", "Medio", "Alto"]  # Zona_Renta: Desconocido tiene menor importancia
])


df_clean_v2[["Potencia", "Averia_grave", "Zona_Renta"]] = ordinal_encoder.fit_transform(df_clean_v2[["Potencia", "Averia_grave", "Zona_Renta"]]).astype(int)

# Confirmar la transformación final
df_clean_v2[["Potencia", "Averia_grave", "Zona_Renta"]].head()


,Potencia,Averia_grave,Zona_Renta
0,0,3,2
1,0,0,2
2,0,0,3
3,0,3,3
4,0,0,4


In [19]:
df_clean_v2.to_csv(os.path.join(processed_dir, "Propensity_clean_v2.csv"), index=False)

# Propensity_clean_v3.csv

En este csv vamos a hacer las siguientes transformaciones
- Cambiar `Zona_Renta`,`Genero`y `Estado_Civil`por la moda
- Eliminar nulos `Averia`

In [20]:
df_clean_v3 = df.copy()

In [21]:
df_clean_v3=df.drop(columns=['CODE'])

In [22]:
# Sustituir valores nulos en 'GENERO', 'ESTADO_CIVIL' y 'Zona_Renta' por su moda
df_clean_v3["GENERO"] = df_clean_v3["GENERO"].fillna(df_clean_v3["GENERO"].mode()[0])
df_clean_v3["ESTADO_CIVIL"] = df_clean_v3["ESTADO_CIVIL"].fillna(df_clean_v3["ESTADO_CIVIL"].mode()[0])
df_clean_v3["Zona_Renta"] = df_clean_v3["Zona_Renta"].fillna(df_clean_v3["Zona_Renta"].mode()[0])


In [23]:
df_clean_v3= df_clean_v3.dropna(subset=['Averia_grave'])

In [24]:
label_cols = ["PRODUCTO", "TIPO_CARROCERIA", "COMBUSTIBLE", "FORMA_PAGO", "ESTADO_CIVIL", 
              "GENERO", "OcupaciOn", "PROVINCIA", "REV_Garantia", "QUEJA_CAC", 
              "Campanna1", "Campanna2", "Campanna3", "Mas_1_coche", "TRANS"]

label_encoders_v3 = {}

for col in label_cols:
    le = LabelEncoder()
    df_clean_v3[col] = le.fit_transform(df_clean_v3[col])
    label_encoders_v3[col] = le  

In [25]:
df_clean_v3["Zona_Renta"].unique()

array(['Medio-Bajo', 'Medio', 'Alto', 'Otros'], dtype=object)

In [26]:
ordinal_encoder = OrdinalEncoder(categories=[
    ["Baja", "Media", "Alta"],  # Potencia: 0, 1, 2
    ["No", "Averia leve", "Averia grave", "Averia muy grave"],  # Averia_grave: 0, 1, 2, 3
    ["Otros", "Medio-Bajo", "Medio", "Alto"] 
])

df_clean_v3[["Potencia", "Averia_grave", "Zona_Renta"]] = ordinal_encoder.fit_transform(
    df_clean_v3[["Potencia", "Averia_grave", "Zona_Renta"]]).astype(int)

In [27]:
df_clean_v3.to_csv(os.path.join(processed_dir, "Propensity_clean_v3.csv"), index=False)

# Propensity_clean_v4

En este csv vamos a hacer las siguientes transformaciones
- Cambiar `Zona_Renta`,`Genero` por la moda
- `Estado_Civil` si es menor de 20 Soltero y si es mayor Casado
- Eliminar nulos `Averia`

In [28]:
df_clean_v4=df.copy()

In [29]:
df_clean_v4 = df_clean_v4.drop(columns=['CODE'])

In [30]:
df_clean_v4 = df_clean_v4.dropna(subset=['Averia_grave'])

In [31]:
# Calcular la moda de la columna Zona_Renta
moda_zona_renta = df_clean_v4['Zona_Renta'].mode()[0]

# Rellenar los valores nulos en la columna 'Zona_Renta' con la moda
df_clean_v4['Zona_Renta'] = df_clean_v4['Zona_Renta'].fillna(moda_zona_renta)

In [32]:
# Rellenar los valores nulos en la columna 'GENERO' con la moda
df['GENERO'] = df['GENERO'].fillna(df['GENERO'].mode()[0])

In [33]:
# Rellenar los valores nulos en la columna 'ESTADO_CIVIL' según la edad del cliente
df_clean_v4['ESTADO_CIVIL'] = df_clean_v4.apply(
    lambda row: 'SOLTERO' if pd.isnull(row['ESTADO_CIVIL']) and row['Edad_Cliente'] < 20 else 
                ('CASADO' if pd.isnull(row['ESTADO_CIVIL']) and row['Edad_Cliente'] >= 20 else row['ESTADO_CIVIL']),
    axis=1
)

In [34]:
label_cols = [
    "PRODUCTO", "TIPO_CARROCERIA", "COMBUSTIBLE", "FORMA_PAGO", "ESTADO_CIVIL", 
    "GENERO", "OcupaciOn", "PROVINCIA", "REV_Garantia", "QUEJA_CAC", 
    "Campanna1", "Campanna2", "Campanna3", "Mas_1_coche", "TRANS"
]

label_encoders_v4 = {}

for col in label_cols:
    le = LabelEncoder()
    df_clean_v4[col] = le.fit_transform(df_clean_v4[col])
    label_encoders_v4[col] = le  # Guardamos el encoder por si necesitamos revertirlo

df_clean_v4[label_cols].head()


,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,REV_Garantia,QUEJA_CAC,Campanna1,Campanna2,Campanna3,Mas_1_coche,TRANS
0,0,0,0,0,0,1,1,4,0,1,1,0,0,0,1
1,0,0,0,0,0,0,1,47,1,0,0,0,0,0,1
2,0,0,0,3,0,1,1,30,0,0,0,0,0,1,1
3,0,0,0,2,0,0,1,32,1,1,1,0,0,1,1
4,0,0,0,2,0,0,2,41,0,0,1,0,1,1,1


In [35]:
ordinal_encoder = OrdinalEncoder(categories=[
    ["Baja", "Media", "Alta"],  # Potencia: 0, 1, 2
    ["No", "Averia leve", "Averia grave", "Averia muy grave"],  # Averia_grave: 0, 1, 2, 3
    ["Otros", "Medio-Bajo", "Medio", "Alto"] 
])

df_clean_v4[["Potencia", "Averia_grave", "Zona_Renta"]] = ordinal_encoder.fit_transform(
    df_clean_v4[["Potencia", "Averia_grave", "Zona_Renta"]]).astype(int)

In [36]:
df_clean_v4.to_csv(os.path.join(processed_dir, "Propensity_clean_v4.csv"), index=False)